In [9]:
class Epidemy(Graph):
    '''Epidemy extends the igraph's class Graph. Ciao
    The additional functionalities are:
    
    1 Built-in getters for graph metrics
        1.1 Plotting
            a)
            b)
        1.2 Metrics
            a) distribution model
            b) eggr
    
    '''
    graph = None
    patient_zero = None
    sentinels = None
    global I
    
    def __init__(self, graph_edge_list, patient_zero = None, sentinels = None):
        '''The compartment label is a byte that can take values 0, 1 and 2, indicating respectivly
        0 - Suscebtible node
        1 - Infected node'''
        
        
        self.graph = Graph.Read_Ncol(graph_edge_list, directed=False)
        self.patient_zero = patient_zero
        self.sentinels = sentinels
             
    
    #setters  
    def setPatientZero(self, patient_zero):
        self.patient_zero = set(patient_zero)
        
    def setSentinels(self, sentinels):
        self.sentinels = set(sentinels)
        
    def resetSentinels(self):
        self.graph.vs["iteration"] = np.nan
        
    def resetNodes(self):
        self.graph.vs["compartment"] = np.zeros(len(self.graph.vs), dtype = np.uint8)
    
     
    #getters
    def getDegreeDistribution(self):
        x,y = np.unique(self.graph.degree(), return_counts=True)
        print(len(self.graph.degree()))
        return (x,y)
    
    def getMaxDegreeVertex(self):
        return self.graph.vs[self.graph.degree().index(self.graph.maxdegree())]
    
    def getSentinelsIteration(self):
        return [i["iteration"] for i in self.graph.vs[self.sentinels]]
    
    def getBestVertexByCentrality(self, number):
        aux = sorted(self.graph.degree())
        return (aux[:number], aux[-number:], "centrality")
        
    def getBestVertexByCloseness(self, number):
        aux = sorted(self.graph.closeness())
        return (aux[:number], aux[-number:], "closeness")
        
    def getBestVertexByBetweennes(self, number):
        aux = sorted(self.graph.pagerank())
        return (aux[:number], aux[-number:], "betweenness")
    
    def getBestVertexByPagerank(self, number):
        aux = sorted(self.graph.betweenness())
        return (aux[:number], aux[-number:], "pagerank")
            
        
    
    #Epidemy Utilities
    def printDegreeDistribution(self, loglog=False):
        '''Prints the degree distirbution of the underlying network using a logarithmic scale'''
        x,y = self.getDegreeDistribution()
        if loglog:
            plt.scatter(np.log(x), np.log(y))
        else:
            plt.scatter(x, y)
        plt.show()
        

            
        
    #Epidemic Models
    def SIR(self, beta = 0.5, mu = 0.1, friend_paradox = False, verbose= False):
        """Simulate an epidemy outbreaks using a sir model"""
        
       
        
        if self.sentinels is None :
            self.sentinels = set([v.index for v in np.random.choice(self.graph.vs, size=5, replace=False)])
            print("No sentinels detected. Random selection: ", self.sentinels)
            
        if friend_paradox:
            I = list(chain.from_iterable([self.graph.neighbors(v) for v in self.graph.vs[self.sentinels]]))
            I = set(np.random.choice(I, size = 5, replace=False))
        elif self.patient_zero is None:
            I = set(np.random.choice(self.graph.vs, size= 5,replace =False))
            print("No patatient zero detected. Random selection: ", self.sentinels)
        else:
            print self.patient_zero
            I = self.patient_zero
        
        
        
        
        self.resetNodes()
        self.resetSentinels();
        
        development = []
        if verbose:
            print("Starting SIR simulation", "beta:", beta, "mu", mu,
              "sentinels:", self.sentinels, "patient_zero: ", self.patient_zero, "friend_paradox", friend_paradox)
        
        for iteration in itertools.count():
            if(len(I)==0):
                print "Epidemy is over... hurray!"
                break
                
            if verbose:   
                print("Nodes in the Infected Compartment: ",len(I))
            
            dI = set([j for j in list(itertools.chain.from_iterable(self.graph.neighborhood(I)))
                      if self.graph.vs[j]["compartment"] == 0 and np.random.random() < beta])
            self.graph.vs[dI]["compartment"] = 1 


            # Finds new removed nodes and update the status
            dR = set([k for k in I if np.random.random() < mu])
            self.graph.vs[dR]["compartment"] = 2
            
            self.graph.vs[dI & set(self.sentinels)]["iteration"] = str(iteration)
            

            I = (I | dI) - (dR)
            
            development +=  [len(I)]
        return development
    

Orkut is a free on-line social network where users form friendship each other. Orkut also allows users form a group which other members can then join. We consider such user-defined groups as ground-truth communities. We provide the Orkut friendship social network and ground-truth communities. This data is provided by Alan Mislove et al.

We regard each connected component in a group as a separate ground-truth community. We remove the ground-truth communities which have less than 3 nodes. We also provide the top 5,000 communities with highest quality which are described in our paper. As for the network, we provide the largest connected component.



The graph is undirected.

In [4]:
from igraph import *
import numpy as np
import matplotlib.pyplot as plt
import itertools
import seaborn as sb
import pandas as pd
from itertools import chain


# Digital Epidemiology Assignment 1

In this homework we are going to use a real social netwrok in order to simulate and analyze a SIR epidemy. 

The network that we are going to use is Orkut's social graph. I decided to use this dataset because Orkut was a community base social network. Since its connections are intrests-based, this would allow us to simulate our empidemy on a network based on a real-social interactions.

Orkut's social graph is undirected.

## Part 1 
 
I decided to implement an extende class of igraph librery called Epidemy. This library is freerly aviable for download from the following link : fefefaeffez

However to make things easier for the correction I included my package in this folder under the name Epidemy.py.

Essentially the epidemy class implements the following functionalities:

* Create Epidemy Object given a Network and an Epidemic Model
* Simulate on the given network the given Epidemic Model
* Get Network metrics
    * Degree Distribution
    * Closeness Distribution
    * Betwenness Distribution
    * 


In this way we can logically separate our code needed to perform some analysis from the analys itself. If the revisor is intrested in the actual implementation of the algorithm she can check the bottom of the ipython notebook to see the implementation.



### 1.1 Compute and Plot a Social Network Degree [X]


In this section we are going to load in memory Orkut and plot its degree distribution.

In [7]:
a = Epidemy('dblp2.txt')
#a.printDegreeDistribution(loglog = True)

### 1.2 SIR Epidemic Model Simulation [X]

Now we are going to simulate a SIR epidemy on the graph:

In [ ]:
epidemic_curve = a.SIR(beta = 0.9, mu = 0.1, patient_zero = [v.index for v in a.graph.vs[-10:]])
plt.plot(epidemic_curve)
#plt.show()

## Part 2

In this section we are going to have a better understaing of the epidemic cycle. More specifically we are going to use epdiemy sentinels, which are a random set o N nodes of the graph which will record the arrival time to them.
Epidemic sentinels are crucial in order to understand the empidemic outbreaks, because are the one that 

### 2.1 SIR Simulation with Static Sentinels [ ] 

In this subsection we are going to exploit the sentinel functionality provided by the class Epidemy. In the previous section we already run a SIR simulation using sentinels, we can retrive the information stored simply accessing the sentinel field of 

In [ ]:
a.setSentinels([v.index for v in a.graph.vs[:10]])
a.SIR(beta = 0.9, mu = 0.1, ver)             
sb.boxplot([-1 if pd.isnull(elem) else int(elem) for elem in seq])
plt.show()

### 2.2 SIR Simulation with Dynamic Random Sentinels, Seeds and Parameters [ ]

Now we want to extend the previous analysis testing different combination of 

* SIR parameters ($\mu$, $\beta$)
* Different sentinels at each loop
* Different number of sentinels
* No overlap between seed and sentinels
* Different seed at each loop
* Multiple Seeds

Most of those features are easily simulable thanks via the built-in functionalities of the epidemy class.

In [ ]:
seq = []
for i in range(2):
    a.SIR(beta = np.random.random(), mu = np.random.random())
    seq = seq + [np.nan if pd.isnull(i["iteration"]) else int(i["iteration"]) for i in a.graph.vs[a.sentinels]]
print seq
sb.boxplot(seq)
plt.show()

In [ ]:
seq = [-1 if pd.isnull(elem) else int(elem) for elem in seq]
seq

### 2.3 SIR Simulation with Static Top-Ranked Sentinels [ ]

Now we want to investigate the impact of specific centrality metrics in the choice of 

In [13]:
def topRankedSentinels(var, ):
    """ We have 8*7 = 56. Some pairs are repeted but since sentinels and seeds are conceptually
    different we count all those combinations combination, we are going to try the those different combinations"""
top = 10
ranks = [a.getBestVertexByCentrality(top), a.getBestVertexByCloseness(top)
         a.getBestVertexByBetweennes(top), a.getBestVertexByPagerank(top)]

for x in ranks:
    for y in ranks:
        print(x[0], y[0]) 
        print(x[0], y[1])
        print(x[1], y[0]) 
        print(x[1], y[1])
        
        
        

    """a.SIR(beta = np.random.random(), mu = np.random.random())
    bestSeq = seq.append([i["iteration"] for i in a.graph.vs[a.sentinels]])
    a.SIR(beta = np.random.random(), mu = np.random.random())
    worstSeq = seq.append([i["iteration"] for i in a.graph.vs[a.sentinels]])"""

    

KeyboardInterrupt: 

In [ ]:
a.getBestVertexByPagerank(top)

In [12]:
print a.getBestVertexByCentrality(10)

([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'centrality')


## Part 3 [ ]

In this section we are intrested in epidemy detection with no global topology information.

### 3.1 Friend Paradox and Local Centrality Measures [ ]

In this subsection we are intrested in minimizing the **detection time** of our epidemy by the sentinels. We suppose that we do not have any information about the global topology of the graph, but only the informations about the neighbours of our seed nodes.

In order to achive this result we are going to exploit the so called **friend paradox**, which we are going to discuss in section 3.2

$\mu=\frac{\sum_{v\in V} d(v)}{|V|}=\frac{2|E|}{|V|}.$

$\frac{\sum_{v\in V} d(v)^2}{2|E|}=\mu + \frac{\sigma^2}{\mu},$

where $ {\sigma}^{2} $ is the variance of the degrees in the graph

In [ ]:
seq = []
for i in range(2):
    a.SIR(beta = 0.9, mu = 0.1, friend_paradox = True , verbose = True)
    seq = seq + [int(i["iteration"]) for i in a.graph.vs[a.sentinels]]
sb.boxplot(np.array(seq))
plt.show()

### 3.2 Why the Friendship Paradox Works*? (and also the local centrality measures) [ ]

The Friendship Paradox states that on average given a node in a graph (that we can imagine as a real person), on average its negihbours (that we can think as its friend) will have an higher degree than the node itself. Exploiting this characteristic is possible to choose from our neighbours the ones that have an higher degree randomly being sure that on average those nodes will have a higher degree, hence an higher porbability to detect the epidemy out break.

We can mix this approach using a also some metrics nalysis an 

## Part 4 (Epidemy Class) [ ]

The epidemy class is the core of this notebook. It extends the igraph graph class adding the methods required to premor an epidemy analysis smoothly